In [1]:
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim

import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [2]:
batch_size = 256
learning_rate = 0.0002
num_epoch = 10

In [3]:
mnist_train = dset.MNIST("./mnist_dataset", train=True, transform=transforms.ToTensor(), target_transform=None, download=True)
mnist_test = dset.MNIST("./mnist_dataset", train=False, transform=transforms.ToTensor(), target_transform=None, download=True)

In [4]:
# Data Loader는 데이터들을 batch_size만큼씩 나누어서 image와 label로 나누어 놓는다.
train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=2, drop_last=True)
test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=False, num_workers=2, drop_last=True)

In [5]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__() # nn.Module의 초기화
        self.layer = nn.Sequential(
            nn.Conv2d(1, 16, 5), # 차례대로 input channel, output channel, kernel size -> w, h = 24
            nn.ReLU(),
            nn.Conv2d(16, 32, 5), # -> w, h = 20
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # -> w, h = 10
            nn.Conv2d(32, 64, 5), # -> w, h = 6
            nn.ReLU(),
            nn.MaxPool2d(2, 2) # -> w, h = 3
        )

        self.fc_layer = nn.Sequential(
            nn.Linear(64*3*3, 100), #64 channel, w, h = 3
            nn.ReLU(),
            nn.Linear(100,10)
        )

    def forward(self, x):
        out = self.layer(x)
        out = out.view(batch_size, -1) # 3d array를 1차원으로 펴준다.
        out = self.fc_layer(out)
        return out

In [6]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [7]:
loss_arr = []
for i in range(num_epoch):
    
    for j, [image, label] in enumerate(train_loader):
        x = image.to(device)
        y_ = label.to(device)

        optimizer.zero_grad()
        output = model.forward(x)
        loss = loss_func(output, y_)
        loss.backward()
        optimizer.step()

        # training data는 총 60,000개이고,
        # batch size가 256이므로
        # batch 갯수는 234개 밖에 안되서 아래 print 구문은 epoch당 1번 밖에 출력되지 않는다.
        if j % 1000 == 0:
            print(loss)
            loss_arr.append(loss.cpu().detach().numpy())


tensor(2.3035, device='cuda:1', grad_fn=<NllLossBackward>)
tensor(0.2785, device='cuda:1', grad_fn=<NllLossBackward>)
tensor(0.1805, device='cuda:1', grad_fn=<NllLossBackward>)
tensor(0.1538, device='cuda:1', grad_fn=<NllLossBackward>)
tensor(0.0897, device='cuda:1', grad_fn=<NllLossBackward>)
tensor(0.0408, device='cuda:1', grad_fn=<NllLossBackward>)
tensor(0.0391, device='cuda:1', grad_fn=<NllLossBackward>)
tensor(0.0416, device='cuda:1', grad_fn=<NllLossBackward>)
tensor(0.0378, device='cuda:1', grad_fn=<NllLossBackward>)
tensor(0.0346, device='cuda:1', grad_fn=<NllLossBackward>)


In [8]:
correct = 0
total = 0
# no_grad를 걸어주는 이유는 test 시에는 back propagation이 필요 없기 때문이다.
with torch.no_grad():
    for image, label in test_loader:
        x = image.to(device)
        y_ = label.to(device)

        output = model.forward(x)
        # 아래 1은 column 방향으로 max를 찾는다는 의미.
        # 즉 여러개의 row 중에서 하나를 고른다.
        # return 값 중 앞의 값은 max value이다.
        _, output_index = torch.max(output, 1)

        total += label.size(0)
        correct += (output_index == y_).sum().float()
    
    print("Accuracy of Test Data: {}".format(100* correct / total))

Accuracy of Test Data: 98.69792175292969
